<a href="https://colab.research.google.com/github/SaidaKarimova9/Emotion-Detection-Intro-to-ML/blob/main/The_Final_version_of_the_second_model_Maximum_Classifier_Discrepancy_(MCD)ipynb.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install numpy pandas matplotlib torch torchvision


  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.whl (176.2 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (99 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.3/21

In [ ]:
from google.colab import drive
drive.mount('/content/drive')



Mounted at /content/drive


In [ ]:
import zipfile
import os
import pandas as pd


In [ ]:
afe_zip_path = '/content/drive/MyDrive/ML-2024/AFE.zip'
expw_zip_path = '/content/drive/MyDrive/ML-2024/ExpW.zip'

with zipfile.ZipFile(afe_zip_path, 'r') as zip_ref:
    zip_ref.extractall('/content/AFE')

with zipfile.ZipFile(expw_zip_path, 'r') as zip_ref:
    zip_ref.extractall('/content/ExpW')


In [ ]:
from torchvision import datasets, transforms

data_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
])

source_train_data = datasets.ImageFolder('/content/AFE/AFE/train', transform=data_transform)
source_test_data = datasets.ImageFolder('/content/AFE/AFE/test', transform=data_transform)

target_train_data = datasets.ImageFolder('/content/ExpW/ExpW/train', transform=data_transform)
target_test_data = datasets.ImageFolder('/content/ExpW/ExpW/test', transform=data_transform)


In [ ]:
from torch.utils.data import DataLoader

batch_size = 32

source_train_loader = DataLoader(source_train_data, batch_size=batch_size, shuffle=True)
source_test_loader = DataLoader(source_test_data, batch_size=batch_size, shuffle=False)

target_train_loader = DataLoader(target_train_data, batch_size=batch_size, shuffle=True)
target_test_loader = DataLoader(target_test_data, batch_size=batch_size, shuffle=False)


In [ ]:
import torch.nn as nn
import torchvision.models as models

class FeatureExtractor(nn.Module):
    def __init__(self):
        super(FeatureExtractor, self).__init__()
        self.backbone = models.resnet50(pretrained=True)
        self.backbone.fc = nn.Identity()

    def forward(self, x):
        features = self.backbone(x)
        return features

class Classifier(nn.Module):
    def __init__(self, num_classes):
        super(Classifier, self).__init__()
        self.fc = nn.Linear(2048, num_classes)

    def forward(self, x):
        out = self.fc(x)
        return out


In [ ]:
import torch
import torch.optim as optim
import torch.nn.functional as F

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

feature_extractor = FeatureExtractor().to(device)
classifier1 = Classifier(num_classes=7).to(device)
classifier2 = Classifier(num_classes=7).to(device)

optimizer_f = optim.Adam(feature_extractor.parameters(), lr=1e-4)
optimizer_c1 = optim.Adam(classifier1.parameters(), lr=1e-4)
optimizer_c2 = optim.Adam(classifier2.parameters(), lr=1e-4)

def train_source():
    feature_extractor.train()
    classifier1.train()
    classifier2.train()

    for epoch in range(10):
        for images, labels in source_train_loader:
            images, labels = images.to(device), labels.to(device)
            features = feature_extractor(images)
            outputs1 = classifier1(features)
            outputs2 = classifier2(features)

            loss1 = F.cross_entropy(outputs1, labels)
            loss2 = F.cross_entropy(outputs2, labels)
            loss = loss1 + loss2

            optimizer_f.zero_grad()
            optimizer_c1.zero_grad()
            optimizer_c2.zero_grad()
            loss.backward()
            optimizer_f.step()
            optimizer_c1.step()
            optimizer_c2.step()

def adapt_target():
    feature_extractor.train()
    classifier1.eval()
    classifier2.eval()

    for epoch in range(10):
        for images, _ in target_train_loader:
            images = images.to(device)
            features = feature_extractor(images)
            outputs1 = classifier1(features)
            outputs2 = classifier2(features)

            discrepancy = torch.mean(torch.abs(F.softmax(outputs1, dim=1) - F.softmax(outputs2, dim=1)))

            optimizer_f.zero_grad()
            discrepancy.backward()
            optimizer_f.step()

train_source()
adapt_target()


/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet50-0676ba61.pth" to /root/.cache/torch/hub/checkpoints/resnet50-0676ba61.pth
100%|██████████| 97.8M/97.8M [00:00<00:00, 193MB/s]


In [ ]:
def evaluate(loader, feature_extractor, classifier):
    feature_extractor.eval()
    classifier.eval()
    correct = 0
    total = 0

    with torch.no_grad():
        for images, labels in loader:
            images, labels = images.to(device), labels.to(device)
            features = feature_extractor(images)
            outputs = classifier(features)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    return 100 * correct / total

source_acc = evaluate(source_test_loader, feature_extractor, classifier1)
target_acc = evaluate(target_test_loader, feature_extractor, classifier1)

print(f'Source Accuracy: {source_acc}%')
print(f'Target Accuracy: {target_acc}%')


Source Accuracy: 48.44522579936271%
Target Accuracy: 40.68055210587943%


In [ ]:
def train_target():
    feature_extractor.train()
    classifier1.train()

    for epoch in range(10):
        for images, labels in target_train_loader:
            images, labels = images.to(device), labels.to(device)
            features = feature_extractor(images)
            outputs = classifier1(features)

            loss = F.cross_entropy(outputs, labels)

            optimizer_f.zero_grad()
            optimizer_c1.zero_grad()
            loss.backward()
            optimizer_f.step()
            optimizer_c1.step()

train_target()

target_oracle_acc = evaluate(target_test_loader, feature_extractor, classifier1)
print(f'Target Oracle Accuracy: {target_oracle_acc}%')


Target Oracle Accuracy: 86.56991803569528%


In [ ]:
results = {
    'Dataset': ['Source (AFE)', 'Target (ExpW)', 'Target Oracle (ExpW)'],
    'Accuracy': [source_acc, target_acc, target_oracle_acc]
}

df = pd.DataFrame(results)
print(df)


                Dataset   Accuracy
0          Source (AFE)  48.445226
1         Target (ExpW)  40.680552
2  Target Oracle (ExpW)  86.569918
